<a href="https://colab.research.google.com/github/Sebastian-Landeo/Predecir-Poblaci-n-Mundial-ML/blob/main/PredecirPoblaci%C3%B3nMundial_MachineLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup
Primero, importemos algunos módulos comunes, asegurémonos de que MatplotLib trace figuras en línea y preparemos una función para guardar las figuras. También verificamos que Python 3.5 o posterior esté instalado (aunque Python 2.x puede funcionar, está obsoleto, por lo que usamos Python 3 en su lugar), así como Scikit-Learn ≥0.20.

In [24]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "end_to_end_project"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

## Configurando Api de Kaggle

In [25]:
api_token = {"username":"sebastiansalcito","key":"b14a94b3a7f6d2f96388b32c98b914bc"}

In [26]:
!rm -rf /root/.kaggle.

In [27]:
!mkdir /root/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [28]:
import json
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

## Consiguiendo la data


In [29]:
!kaggle datasets list

ref                                                             title                                             size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  -----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
shreyanshverma27/online-sales-dataset-popular-marketplace-data  Online Sales Dataset - Popular Marketplace Data    7KB  2024-05-25 23:55:26          10634        193  1.0              
nelgiriyewithana/most-streamed-spotify-songs-2024               Most Streamed Spotify Songs 2024                 496KB  2024-06-15 18:50:51           2275         59  1.0              
zeesolver/data-eng-salary-2024                                  Data Developer Salary in 2024💰                   108KB  2024-06-10 07:33:22           2332         31  1.0              
rabieelkharoua/students-performance-dataset                     📚 Students 

In [30]:
!chmod 600 /root/.kaggle/kaggle.json

In [31]:
!kaggle datasets download -d mjdskaggle/2024-population-projections-by-country


Dataset URL: https://www.kaggle.com/datasets/mjdskaggle/2024-population-projections-by-country
License(s): MIT
2024-population-projections-by-country.zip: Skipping, found more recently modified local copy (use --force to force download)


In [32]:
import zipfile
archivo_zip = '/content/2024-population-projections-by-country.zip'
with zipfile.ZipFile('/content/2024-population-projections-by-country.zip', 'r') as zip_file:
  for nombre_archivo in zip_file.namelist():
    print(nombre_archivo)

2024Populations.csv


## Dando un vistazo rápido a la información

In [33]:
import pandas as pd
data_com = pd.read_csv('/content/2024-population-projections-by-country.zip')
data_com.head()

,rank,country,TwoLetterID,unMember,pop1980,pop2000,pop2010,pop2023,pop2024,pop2030,pop2050,landAreaKm,2024YoYChange,2024YoYGrowthRate,2024WorldPercentage,Density_2024
0,1,India,IN,True,696828385,1059633675,1240613620,1428627663,1441719852,1514994080,1670490596,2973190.0,13092189,0.009164,0.177614,484.906734
1,2,China,CN,True,982372466,1264099069,1348191368,1425671352,1425178782,1415605906,1312636325,9424702.9,-492570,-0.000346,0.175577,151.217370
2,3,United States,US,True,223140018,282398554,311182845,339996563,341814420,352162301,375391963,9147420.0,1817857,0.005347,0.042110,37.367304
3,4,Indonesia,ID,True,148177096,214072421,244016173,277534122,279798049,292150100,317225213,1877519.0,2263927,0.008157,0.034470,149.025416
4,5,Pakistan,PK,True,80624057,154369924,194454498,240485658,245209815,274029836,367808468,770880.0,4724157,0.019644,0.030209,318.090773


**Revisamos si los datos son numéricos o categóricos**

En este vistazo se puede ver:
- Que TwoLetterID y contry pueden ser categóricos por ser de tipo object. Pero estos son identificadores y nombres, es decir, no clasifican nada.
- TwolletterId le falta 1 dato a comparación de los demás

In [34]:
data_com.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234 entries, 0 to 233
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   rank                 234 non-null    int64  
 1   country              234 non-null    object 
 2   TwoLetterID          233 non-null    object 
 3   unMember             234 non-null    bool   
 4   pop1980              234 non-null    int64  
 5   pop2000              234 non-null    int64  
 6   pop2010              234 non-null    int64  
 7   pop2023              234 non-null    int64  
 8   pop2024              234 non-null    int64  
 9   pop2030              234 non-null    int64  
 10  pop2050              234 non-null    int64  
 11  landAreaKm           234 non-null    float64
 12  2024YoYChange        234 non-null    int64  
 13  2024YoYGrowthRate    234 non-null    float64
 14  2024WorldPercentage  234 non-null    float64
 15  Density_2024         234 non-null    flo

**Iterpretación de value_counts:**

Actualización: Estos pasos ya no son necesarios, por lo dicho arriba sobre el identificador y nombres.
La columna country es de tipo object, lo que generalmente indica que contiene datos de texto.

Frecuencia de Valores: La función value_counts() cuenta la cantidad de ocurrencias de cada valor único en la columna country. En este caso, cada país mencionado tiene exactamente 1 ocurrencia.

Número Total de Valores Únicos (length 234): Hay un total de 234 valores únicos en la columna country.

El tipo de datos del resultado (dtype): es int64, indicando que las cuentas son enteros de 64 bits. el tipo de dato int64 se refiere a los conteos de las ocurrencias, no a los nombres de los países.


In [35]:
data_com["country"].value_counts()

country
India                 1
Reunion               1
Slovenia              1
North Macedonia       1
Latvia                1
                     ..
Dominican Republic    1
Jordan                1
South Sudan           1
Cuba                  1
Vatican City          1
Name: count, Length: 234, dtype: int64

In [36]:
data_com["TwoLetterID"].value_counts()

TwoLetterID
IN    1
FJ    1
MK    1
LV    1
GQ    1
     ..
DO    1
JO    1
SS    1
CU    1
VA    1
Name: count, Length: 233, dtype: int64

## Analizando los estadísticos

In [37]:
data_com.describe()

,rank,pop1980,pop2000,pop2010,pop2023,pop2024,pop2030,pop2050,landAreaKm,2024YoYChange,2024YoYGrowthRate,2024WorldPercentage,Density_2024
count,234.000000,2.340000e+02,2.340000e+02,2.340000e+02,2.340000e+02,2.340000e+02,2.340000e+02,2.340000e+02,2.340000e+02,2.340000e+02,234.000000,2.340000e+02,234.000000
mean,117.500000,1.898462e+07,2.626947e+07,2.984524e+07,3.437443e+07,3.468862e+07,3.651461e+07,4.148628e+07,5.571123e+05,3.141905e+05,0.009135,4.273504e-03,456.125781
std,67.694165,8.178519e+07,1.116982e+08,1.242185e+08,1.373864e+08,1.380750e+08,1.417827e+08,1.481676e+08,1.689972e+06,1.075131e+06,0.011379,1.701031e-02,1991.330932
min,1.000000,7.330000e+02,6.510000e+02,5.960000e+02,7.640000e+02,7.640000e+02,7.640000e+02,7.640000e+02,4.400000e-01,-8.043410e+05,-0.030870,9.412187e-08,0.138358
25%,59.250000,2.296142e+05,3.272420e+05,3.931490e+05,4.225982e+05,4.264565e+05,4.561490e+05,5.466058e+05,2.625875e+03,1.975000e+02,0.001928,5.253780e-05,39.801183
50%,117.500000,3.141146e+06,4.292907e+06,4.942770e+06,5.643895e+06,5.626359e+06,6.178231e+06,6.352397e+06,7.568925e+04,1.997850e+04,0.007786,6.931458e-04,98.287400
75%,175.750000,9.826054e+06,1.576230e+07,1.915957e+07,2.324537e+07,2.392272e+07,2.616311e+07,3.568614e+07,4.047876e+05,2.238608e+05,0.015670,2.947187e-03,248.223177
max,234.000000,9.823725e+08,1.264099e+09,1.348191e+09,1.428628e+09,1.441720e+09,1.514994e+09,1.670491e+09,1.637687e+07,1.309219e+07,0.048264,1.776143e-01,21674.224920


## Mostrar histogramas
La población tiene escala de $1e^9$

Se concluye que los datos más importante serán: la población de 1980, 2000, 2010; y la tasa de crecimiento. Cabe destacar que las poblacióne tiene datos outliers.

In [38]:
# Importar las bibliotecas necesarias
import pandas as pd
import matplotlib.pyplot as plt

# Supongamos que 'data_com' es un DataFrame ya cargado con los datos adecuados
# data_com = pd.read_csv("ruta_a_tu_archivo.csv")

# Definir la función save_fig
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = fig_id + "." + fig_extension
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
    print(f"Guardado {path}")

# Filtrar las columnas numéricas
numeric_data = data_com.select_dtypes(include=[int, float])

# Crear histogramas con configuraciones personalizadas para el tamaño del texto
fig, axs = plt.subplots(nrows=(len(numeric_data.columns) // 2 + len(numeric_data.columns) % 2), ncols=2, figsize=(60, 100))

axs = axs.flatten()  # Aplanar la matriz de ejes para facilitar la iteración

for i, col in enumerate(numeric_data.columns):
    numeric_data[col].hist(bins=50, ax=axs[i])
    axs[i].set_title(col, fontsize=40)  # Títulos más grandes
    axs[i].set_xlabel('Valor', fontsize=32)  # Etiquetas del eje x más grandes
    axs[i].set_ylabel('Frecuencia', fontsize=32)  # Etiquetas del eje y más grandes
    axs[i].tick_params(axis='x', labelsize=28)  # Tamaño de las etiquetas del eje x
    axs[i].tick_params(axis='y', labelsize=28)  # Tamaño de las etiquetas del eje y

# Ajustar el diseño para evitar la superposición
plt.tight_layout()

# Guardar la figura
save_fig("attribute_histogram_plots")

# Mostrar los gráficos
plt.show()


Guardado attribute_histogram_plots.png


## Crear un conjunto de prueba

In [ ]:
# to make this notebook's output identical at every run
np.random.seed(2001)

Dividir un conjunto de datos en dos subconjuntos: uno para entrenamiento y otro para prueba.